#### Read multiple las file at the same time
#### Create different matrices to represent different classes
1. Unclassified

(green)
2. Bare-earth and low grass
3. Low vegetation (height < 2m)
4. Hight vegetation (height > 2m)

(blue)
5. Water

6. Buildings
7. Other
8. Noise (noise points, blunders, outliners, etc)





In [1]:
import numpy as np

In [2]:
import laspy

In [3]:
import open3d as o3d

In [4]:
las = laspy.read("/Users/peinanwang/VancouverLAS/sample_data/downtownLAS/downtown.las")

In [5]:
las

<LasData(1.2, point fmt: <PointFormat(1, 0 bytes of extra dims)>, 39977102 points, 1 vlrs)>

In [6]:
las.header

<LasHeader(1.2, <PointFormat(1, 0 bytes of extra dims)>)>

In [7]:
las.header.point_format

<PointFormat(1, 0 bytes of extra dims)>

In [8]:
list(las.point_format.dimension_names)

['X',
 'Y',
 'Z',
 'intensity',
 'return_number',
 'number_of_returns',
 'scan_direction_flag',
 'edge_of_flight_line',
 'classification',
 'synthetic',
 'key_point',
 'withheld',
 'scan_angle_rank',
 'user_data',
 'point_source_id',
 'gps_time']

In [9]:
las.X

array([9101008, 9101138, 9103472, ..., 9199990, 9199893, 9199814],
      dtype=int32)

In [10]:
las.intensity

array([ 5985,  8606, 21976, ..., 17563, 20272,  8781], dtype=uint16)

In [11]:
las.classification

<SubFieldView([7 7 7 ... 1 1 1])>

In [12]:
las.synthetic

<SubFieldView([0 0 0 ... 0 0 0])>

In [13]:
points_with_classification = np.stack([las.X, las.Y, las.Z, las.classification], axis=0).transpose(1, 0)

In [14]:
points_with_classification

type(points_with_classification)

for element in points_with_classification[0]:
    print(type(element))
    
points_with_classification[0][3] == 5

<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>
<class 'numpy.int32'>


False

### create different matrices based on classification

In [15]:
'''
Detail for LAS classification code can be found at:
https://desktop.arcgis.com/en/arcmap/10.3/manage-data/las-dataset/lidar-point-classification.htm

Code 8 and 12 will not be used for this project
'''

LAS_CLASSIFICATION = {0 : "Never Classified", 
                      1 : "Unassigned",
                      2 : "Ground",
                      3 : "Low Vegetation",
                      4 : "Medium Vegetation",
                      5 : "High Vegetation", 
                      6 : "Building",
                      7 : "Noise",
                      8 : "Model Key/Reserved", 
                      9 : "Water", 
                      10: "Rail",
                      11: "Road Surface",
                      12: "Overlap/Reserved",
                      13: "Wire - Guard", 
                      14: "Wire - Conductor", 
                      15: "Transmission Tower",
                      16: "Wire - Connector", 
                      17: "Bridge Deck", 
                      18: "High Noise"}

CLASSIFICATION_COLORS = {"Never Classified" : [204, 255, 255],# light blue
                         "Unassigned" : [255, 204, 255],      # light pink
                         "Ground" : [204, 153, 0],            # light brown
                         "Low Vegetation" : [153, 255, 51],   # light green
                         "Medium Vegetation" : [51, 204, 51], # green
                         "High Vegetation" : [51, 153, 51],   # dark green
                         "Building" : [255, 102, 0],          # orange
                         "Noise" : [255, 255, 204],           # light yellow (almost white)
                         "Model Key/Reserved" : [51, 51, 0],  # dark green (almost black)       
                         "Water" : [0, 102, 255],             # blue
                         "Rail" : [0, 0, 102],                # dark blue
                         "Road Surface" : [102, 102, 153],    # dark grey
                         "Overlap/Reserved" : [51, 51, 0],    # dark green (almost black)  
                         "Wire - Guard" : [204, 0, 204],      # purple
                         "Wire - Conductor" : [204, 0, 204],  # purple
                         "Transmission Tower" : [204, 0, 204],# purple
                         "Wire - Connector" : [204, 0, 204],  # purple
                         "Bridge Deck" : [255, 0, 0],         # red
                         "High Noise" : [255, 255, 204]       # light yellow (almost white)
                        }

NATURAL_FEATURES = [3, 4, 5]

POWERLINES = [13, 14, 16]

UNNATURAL_STRUCTURES = [6, 11, 17]




In [16]:
point_cloud_matrices = []

for key in LAS_CLASSIFICATION:
    point_cloud_matrices.append([])

print(point_cloud_matrices)

for point in points_with_classification:
    i = point[3]
    point_cloud_matrices[i].append([point[0], point[1], point[2]])

n = 0
for arr in point_cloud_matrices:
    print(f"{n} {LAS_CLASSIFICATION[n]}: {len(arr)}")
    n += 1

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
0 Never Classified: 0
1 Unassigned: 9662679
2 Ground: 3880901
3 Low Vegetation: 670014
4 Medium Vegetation: 0
5 High Vegetation: 6146149
6 Building: 19541588
7 Noise: 75771
8 Model Key/Reserved: 0
9 Water: 0
10 Rail: 0
11 Road Surface: 0
12 Overlap/Reserved: 0
13 Wire - Guard: 0
14 Wire - Conductor: 0
15 Transmission Tower: 0
16 Wire - Connector: 0
17 Bridge Deck: 0
18 High Noise: 0


In [17]:
classification_set = set(list(las.classification))

KeyboardInterrupt: 

In [ ]:
print(classification_set)

### create geom object for each classification
### save geom objects to a list

In [ ]:
RGB = 255

In [ ]:
geom_list = []
for i in range(len(point_cloud_matrices)):
    geom = o3d.geometry.PointCloud()
    geom.points = o3d.utility.Vector3dVector(point_cloud_matrices[i])
    color = CLASSIFICATION_COLORS[LAS_CLASSIFICATION[i]]
    geom.paint_uniform_color([color[0]/RGB, color[1]/RGB, color[2]/RGB])
    geom_list.append(geom)

In [ ]:
o3d.visualization.draw_geometries(geom_list)